In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import gc, math

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

##### Set the size and styles of graphs

In [2]:
sns.set(rc={'figure.figsize':(11,8)})
sns.set(style="whitegrid")

### Read the dataset
Data is given in different CSV files which we will need to merge afterwards. 

`train.csv` only contains the ID of the building and meter related information including our target variable to be predicted (`meter_reading`). This `building_id` is foreign key in `building_metadata.csv`. All the information related to the buildings are given in this file. 


Same goes for `weather_train.csv` and `building_metadata.csv` files with common column (foreign key) `site_id`. So all three files are related and we will have to join these tables later

In [3]:
%%time
metadata_df = pd.read_csv('../raw_data/building_metadata.csv')

CPU times: user 2.81 ms, sys: 908 µs, total: 3.71 ms
Wall time: 3.46 ms


In [4]:
train_df = pd.read_csv('../raw_data/train.csv', parse_dates=['timestamp'])

In [5]:
test_df = pd.read_csv('../raw_data/test.csv', parse_dates=['timestamp'])

In [6]:
weather_train_df = pd.read_csv('../raw_data/weather_train.csv', parse_dates=['timestamp'])

In [7]:
weather_test_df = pd.read_csv('../raw_data/weather_test.csv', parse_dates=['timestamp'])

In [8]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [9]:
weather_train_df.shape

(139773, 9)

In [10]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


### Align timestamps
Timestap data is not in their local time. As energy consumptions are related to the local time, an alighment is nescessary before using timestamp. 

The credit goes to [this kernel](https://www.kaggle.com/nz0722/aligned-timestamp-lgbm-by-meter-type) for the idea. Refer it for more details and explanation about below code.

In [11]:
weather = pd.concat([weather_train_df,weather_test_df],ignore_index=True)
weather_key = ['site_id', 'timestamp']

temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

# calculate ranks of hourly temperatures within date/site_id chunks
temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')

# create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

# Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
site_ids_offsets.index.name = 'site_id'

def timestamp_align(df):
    df['offset'] = df.site_id.map(site_ids_offsets)
    df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
    df['timestamp'] = df['timestamp_aligned']
    del df['timestamp_aligned']
    return df

In [12]:
weather_train_df.tail()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6
139772,15,2016-12-31 23:00:00,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8


In [13]:
weather_train_df = timestamp_align(weather_train_df)

In [14]:
weather_test_df = timestamp_align(weather_test_df)

In [15]:
del weather 
del df_2d
del temp_skeleton
del site_ids_offsets

In [16]:
weather_train_df.tail()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset
139768,15,2016-12-31 13:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7,6
139769,15,2016-12-31 14:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7,6
139770,15,2016-12-31 15:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1,6
139771,15,2016-12-31 16:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6,6
139772,15,2016-12-31 17:00:00,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,6


### Fill NaNs in weather data by interpolation

In [17]:
weather_train_df.isna().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
offset                    0
dtype: int64

In [18]:
weather_test_df.isna().sum()

site_id                    0
timestamp                  0
air_temperature          104
cloud_coverage        140448
dew_temperature          327
precip_depth_1_hr      95588
sea_level_pressure     21265
wind_direction         12370
wind_speed               460
offset                     0
dtype: int64

In [19]:
weather_train_df['air_temperature'] = weather_train_df[['site_id','air_temperature' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_train_df['cloud_coverage']=weather_train_df[['site_id','cloud_coverage' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_train_df['dew_temperature' ]=weather_train_df[['site_id','dew_temperature' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_train_df['precip_depth_1_hr']=weather_train_df[['site_id','precip_depth_1_hr' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_train_df['sea_level_pressure']=weather_train_df[['site_id','sea_level_pressure' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]
weather_train_df['wind_direction']=weather_train_df[['site_id','wind_direction' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_train_df['wind_speed' ]=weather_train_df[['site_id','wind_speed' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

/var/folders/99/gh518qd1231cp1cqx_b4d36w0000gn/T/ipykernel_29780/1687896302.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  weather_train_df['air_temperature'] = weather_train_df[['site_id','air_temperature' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]
/var/folders/99/gh518qd1231cp1cqx_b4d36w0000gn/T/ipykernel_29780/1687896302.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

In [20]:
weather_test_df['air_temperature'] = weather_test_df[['site_id','air_temperature' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_test_df['cloud_coverage']=weather_test_df[['site_id','cloud_coverage' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_test_df['dew_temperature' ]=weather_test_df[['site_id','dew_temperature' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_test_df['precip_depth_1_hr']=weather_test_df[['site_id','precip_depth_1_hr' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_test_df['sea_level_pressure']=weather_test_df[['site_id','sea_level_pressure' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]
weather_test_df['wind_direction']=weather_test_df[['site_id','wind_direction' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

weather_test_df['wind_speed' ]=weather_test_df[['site_id','wind_speed' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]

/var/folders/99/gh518qd1231cp1cqx_b4d36w0000gn/T/ipykernel_29780/3573677034.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  weather_test_df['air_temperature'] = weather_test_df[['site_id','air_temperature' ]].groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both')).iloc[:,1]
/var/folders/99/gh518qd1231cp1cqx_b4d36w0000gn/T/ipykernel_29780/3573677034.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use



In [21]:
weather_train_df.isna().sum()

site_id                   0
timestamp                 0
air_temperature           0
cloud_coverage        17228
dew_temperature           0
precip_depth_1_hr     26273
sea_level_pressure     8755
wind_direction            0
wind_speed                0
offset                    0
dtype: int64

In [22]:
weather_test_df.isna().sum()

site_id                   0
timestamp                 0
air_temperature           0
cloud_coverage        33146
dew_temperature           0
precip_depth_1_hr     51807
sea_level_pressure    17241
wind_direction            0
wind_speed                0
offset                    0
dtype: int64

In [23]:
train_df['meter_reading'] = np.log1p(train_df['meter_reading'])

In [24]:
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset
0,0,2015-12-31 19:00:00,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,5
1,0,2015-12-31 20:00:00,24.4,4.0,21.1,-1.0,1020.2,70.0,1.5,5
2,0,2015-12-31 21:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,5
3,0,2015-12-31 22:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,5
4,0,2015-12-31 23:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,5


In [25]:
## Function to reduce the memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [26]:
le = LabelEncoder()
metadata_df['primary_use'] = le.fit_transform(metadata_df['primary_use'])

In [27]:
metadata_df = reduce_mem_usage(metadata_df)
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

Mem. usage decreased to  0.02 Mb (74.9% reduction)
Mem. usage decreased to 250.63 Mb (59.4% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)


In [28]:
weather_train_df = reduce_mem_usage(weather_train_df)

Mem. usage decreased to  3.20 Mb (70.0% reduction)


In [29]:
weather_test_df = reduce_mem_usage(weather_test_df)

Mem. usage decreased to  6.35 Mb (70.0% reduction)


#### Get insights of shapes and first few data rows of all the files

In [30]:
print (f'Training data shape: {train_df.shape}')
print (f'Weather training shape: {weather_train_df.shape}')
print (f'Weather training shape: {weather_test_df.shape}')
print (f'Weather testing shape: {metadata_df.shape}')
print (f'Test data shape: {test_df.shape}')

Training data shape: (20216100, 4)
Weather training shape: (139773, 10)
Weather training shape: (277243, 10)
Weather testing shape: (1449, 6)
Test data shape: (41697600, 4)


In [31]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [32]:
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset
0,0,2015-12-31 19:00:00,25.000000,6.0,20.00000,-1.0,1019.5,0.0,0.000000,5
1,0,2015-12-31 20:00:00,24.406250,4.0,21.09375,-1.0,1020.0,70.0,1.500000,5
2,0,2015-12-31 21:00:00,22.796875,2.0,21.09375,0.0,1020.0,0.0,0.000000,5
3,0,2015-12-31 22:00:00,21.093750,2.0,20.59375,0.0,1020.0,0.0,0.000000,5
4,0,2015-12-31 23:00:00,20.000000,2.0,20.00000,-1.0,1020.0,250.0,2.599609,5


In [33]:
metadata_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,7432,2008.0,NaN
1,0,1,0,2720,2004.0,NaN
2,0,2,0,5376,1991.0,NaN
3,0,3,0,23685,2002.0,NaN
4,0,4,0,116607,1975.0,NaN


In [34]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


#### Merge necessary files
As mentioned previously, to get a single dataframe for training and a single data frame for testing with all the feature included, we need to join the tables/files which are related by foreign keys. Let's first merge/join training data.

In [35]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [36]:
%%time
full_train_df = train_df.merge(metadata_df, on='building_id', how='left')
full_train_df = full_train_df.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')

CPU times: user 3.96 s, sys: 1.73 s, total: 5.69 s
Wall time: 5.9 s


Drop all NaN rows which are generated by timestamp alignment

In [37]:
full_train_df = full_train_df.loc[~(full_train_df['air_temperature'].isnull() & full_train_df['cloud_coverage'].isnull() & full_train_df['dew_temperature'].isnull() & full_train_df['precip_depth_1_hr'].isnull() & full_train_df['sea_level_pressure'].isnull() & full_train_df['wind_direction'].isnull() & full_train_df['wind_speed'].isnull() & full_train_df['offset'].isnull())]

In [38]:
full_train_df.shape

(20112649, 17)

#### Reducing the memory usage
Let's delete unnecessary dataframes from memory to lower the memory usage

In [39]:
# Delete unnecessary dataframes to decrease memory usage
del train_df
del weather_train_df
gc.collect()

23

Now let's change the data types of necessary feature columns based on the range of the data values. This will lower the data usage. But **how**? Let's see. For example the datatype of feature `building_id` is `int64` but based on the range of this feature, it can be accomodated in lower range i.e. `int16`. So this will decrease the memory usage.

#### Let's do the same for test data

In [40]:
%%time
full_test_df = test_df.merge(metadata_df, on='building_id', how='left')
full_test_df = full_test_df.merge(weather_test_df, on=['site_id', 'timestamp'], how='left')

CPU times: user 8.28 s, sys: 4.05 s, total: 12.3 s
Wall time: 13 s


In [41]:
full_test_df.shape

(41697600, 17)

#### Reducing the memory usage
Let's delete unnecessary dataframes from memory to lower the memory usage

In [42]:
# Delete unnecessary dataframes to decrease memory usage
del metadata_df
del weather_test_df
del test_df
gc.collect()

0

Now let's change the data types of necessary feature columns based on the range of the data values. This will lower the data usage. But **how**? Let's see. For example the datatype of feature `building_id` is `int64` but based on the range of this feature, it can be accomodated in lower range i.e. `int16`. So this will decrease the memory usage.

In [43]:
full_train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset
0,0,0,2016-01-01,0.0,0,0,7432,2008.0,NaN,19.40625,4.0,19.40625,0.0,1019.5,0.0,0.0,5.0
1,1,0,2016-01-01,0.0,0,0,2720,2004.0,NaN,19.40625,4.0,19.40625,0.0,1019.5,0.0,0.0,5.0
2,2,0,2016-01-01,0.0,0,0,5376,1991.0,NaN,19.40625,4.0,19.40625,0.0,1019.5,0.0,0.0,5.0
3,3,0,2016-01-01,0.0,0,0,23685,2002.0,NaN,19.40625,4.0,19.40625,0.0,1019.5,0.0,0.0,5.0
4,4,0,2016-01-01,0.0,0,0,116607,1975.0,NaN,19.40625,4.0,19.40625,0.0,1019.5,0.0,0.0,5.0


## Feature Engineering
The joined dataframe (full_train_df) now has 20,216,100 rows, and 16 features in training dataset.

### Analysing missing data
First let's count and fill missing data in training datasets

In [44]:
pd.DataFrame(full_train_df.isna().sum().sort_values(ascending=False), columns=['NaN Count'])

,NaN Count
floor_count,16618674
year_built,12105206
precip_depth_1_hr,1646098
sea_level_pressure,779195
cloud_coverage,476553
air_temperature,0
wind_speed,0
wind_direction,0
dew_temperature,0
building_id,0


Missing data can be filled in many ways. Here are few techniques to fill missing values: 

* Ignore the data row
* Back-fill or forward-fill to propagate next or previous values respectively
* Replace with some constant value outside fixed value range-999,-1 etc.
* Replace with mean, median value

For now, we will go with last method. So let's fill all the missing data with it's average(mean) values of corresponding columns.

In [45]:
def mean_without_overflow_fast(col):
    col /= len(col)
    return col.mean() * len(col)

In [46]:
missing_values = (100-full_train_df.count() / len(full_train_df) * 100).sort_values(ascending=False)

In [47]:
missing_values

floor_count           82.627972
year_built            60.187030
precip_depth_1_hr      8.184392
sea_level_pressure     3.874154
cloud_coverage         2.369419
air_temperature        0.000000
wind_speed             0.000000
wind_direction         0.000000
dew_temperature        0.000000
building_id            0.000000
meter                  0.000000
square_feet            0.000000
primary_use            0.000000
site_id                0.000000
meter_reading          0.000000
timestamp              0.000000
offset                 0.000000
dtype: float64

In [48]:
%%time
missing_features = full_train_df.loc[:, missing_values > 0.0]
missing_features = missing_features.apply(mean_without_overflow_fast)

CPU times: user 600 ms, sys: 391 ms, total: 991 ms
Wall time: 990 ms


In [49]:
missing_features

year_built            1968.344315
floor_count              4.175326
cloud_coverage           2.750034
precip_depth_1_hr        1.255560
sea_level_pressure    1016.059575
dtype: float64

In [50]:
#wp = weather_test_df

In [51]:
#wp['cloud_coverage'] = wp.cloud_coverage.fillna(value='2.750034')
#wp['precip_depth_1_hr'] = wp.precip_depth_1_hr.fillna(value='1.255560')
#wp['sea_level_pressure'] = wp.sea_level_pressure.fillna(value='1016.059575')

In [52]:
#wp.to_csv("../raw_data/weather_preproc.csv")

In [53]:
for key in full_train_df.loc[:, missing_values > 0.0].keys():
    if key == 'year_built' or key == 'floor_count':
        full_train_df[key].fillna(math.floor(missing_features[key]), inplace=True)
        full_test_df[key].fillna(math.floor(missing_features[key]), inplace=True)
    else:
        full_train_df[key].fillna(missing_features[key], inplace=True)
        full_test_df[key].fillna(missing_features[key], inplace=True)

In [54]:
full_train_df.tail()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset
20215251,1064,0,2016-12-31 23:00:00,5.042969,12,0,131212,1968.0,4.0,6.5,8.0,5.300781,1.25556,1019.5,340.0,6.0,0.0
20215252,1065,0,2016-12-31 23:00:00,1.827148,12,6,12152,1968.0,4.0,6.5,8.0,5.300781,1.25556,1019.5,340.0,6.0,0.0
20215253,1066,0,2016-12-31 23:00:00,3.291016,12,0,55800,1968.0,4.0,6.5,8.0,5.300781,1.25556,1019.5,340.0,6.0,0.0
20215254,1067,0,2016-12-31 23:00:00,5.445312,12,0,125701,1968.0,4.0,6.5,8.0,5.300781,1.25556,1019.5,340.0,6.0,0.0
20215255,1068,0,2016-12-31 23:00:00,4.050781,12,0,112956,1968.0,4.0,6.5,8.0,5.300781,1.25556,1019.5,340.0,6.0,0.0


In [55]:
full_test_df.tail()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset
41697595,41697595,1444,0,2018-05-09 07:00:00,15,1,19619,1914.0,4.0,18.90625,0.0,10.601562,-1.0,1020.5,180.0,2.599609,6.0
41697596,41697596,1445,0,2018-05-09 07:00:00,15,0,4298,1968.0,4.0,18.90625,0.0,10.601562,-1.0,1020.5,180.0,2.599609,6.0
41697597,41697597,1446,0,2018-05-09 07:00:00,15,1,11265,1997.0,4.0,18.90625,0.0,10.601562,-1.0,1020.5,180.0,2.599609,6.0
41697598,41697598,1447,0,2018-05-09 07:00:00,15,4,29775,2001.0,4.0,18.90625,0.0,10.601562,-1.0,1020.5,180.0,2.599609,6.0
41697599,41697599,1448,0,2018-05-09 07:00:00,15,6,92271,2001.0,4.0,18.90625,0.0,10.601562,-1.0,1020.5,180.0,2.599609,6.0


So all the missing values for training and testing data is now filled with the mean of corresponding feature columns.

### Adding few more features

First let's expand timestamp to multiple components

In [56]:
full_train_df['timestamp'].dtype

dtype('<M8[ns]')

In [57]:
full_train_df["timestamp"] = pd.to_datetime(full_train_df["timestamp"])
full_test_df["timestamp"] = pd.to_datetime(full_test_df["timestamp"])

In [58]:
def transform(df):
    df['hour'] = np.uint8(df['timestamp'].dt.hour)
    df['day'] = np.uint8(df['timestamp'].dt.day)
    df['weekday'] = np.uint8(df['timestamp'].dt.weekday)
    df['month'] = np.uint8(df['timestamp'].dt.month)
    df['year'] = np.uint8(df['timestamp'].dt.year-1900)
    
    df['square_feet'] = np.log(df['square_feet'])
    
    return df

In [59]:
full_train_df = transform(full_train_df)
full_test_df = transform(full_test_df)

In [60]:
def encode_cyclic_feature(df, col, max_vals):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_vals)
#     df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_vals)
    del df[col]
    return df

In [61]:
full_train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset,hour,day,weekday,month,year
0,0,0,2016-01-01,0.0,0,0,8.913550,2008.0,4.0,19.40625,...,0.0,1019.5,0.0,0.0,5.0,0,1,4,1,116
1,1,0,2016-01-01,0.0,0,0,7.908387,2004.0,4.0,19.40625,...,0.0,1019.5,0.0,0.0,5.0,0,1,4,1,116
2,2,0,2016-01-01,0.0,0,0,8.589700,1991.0,4.0,19.40625,...,0.0,1019.5,0.0,0.0,5.0,0,1,4,1,116
3,3,0,2016-01-01,0.0,0,0,10.072597,2002.0,4.0,19.40625,...,0.0,1019.5,0.0,0.0,5.0,0,1,4,1,116
4,4,0,2016-01-01,0.0,0,0,11.666565,1975.0,4.0,19.40625,...,0.0,1019.5,0.0,0.0,5.0,0,1,4,1,116


In [62]:
dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
full_train_df['is_holiday'] = (full_train_df['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
full_test_df['is_holiday'] = (full_test_df['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

In [63]:
# Assuming 5 days a week for all the given buildings
full_train_df.loc[(full_train_df['weekday'] == 5) | (full_train_df['weekday'] == 6) , 'is_holiday'] = 1
full_test_df.loc[(full_test_df['weekday']) == 5 | (full_test_df['weekday'] == 6) , 'is_holiday'] = 1

In [64]:
# full_train_df.loc[(full_train_df['primary_use'] == le.transform(['Education'])[0]) & (full_train_df['month'] >= 6) & (full_train_df['month'] <= 8), 'is_vacation_month'] = np.int8(1)
# full_train_df.loc[full_train_df['is_vacation_month']!=1, 'is_vacation_month'] = np.int8(0)

### Removing weired data on site_id 0
As you can see above, this data looks weired until May 20. It is reported in this discussion by @barnwellguy that All electricity meter is 0 until May 20 for site_id == 0. Let's remove these data from training data.

In [65]:
full_train_df.shape

(20112649, 23)

In [66]:
full_train_df = full_train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [67]:
full_train_df.shape

(19766435, 23)

In [68]:
full_test_df = full_test_df.drop(['timestamp'], axis=1)
full_train_df = full_train_df.drop(['timestamp'], axis=1)
print (f'Shape of training dataset: {full_train_df.shape}')
print (f'Shape of testing dataset: {full_test_df.shape}')

Shape of training dataset: (19766435, 22)
Shape of testing dataset: (41697600, 22)


In [69]:
full_train_df.tail()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,sea_level_pressure,wind_direction,wind_speed,offset,hour,day,weekday,month,year,is_holiday
20215251,1064,0,5.042969,12,0,11.784570,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1
20215252,1065,0,1.827148,12,6,9.405249,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1
20215253,1066,0,3.291016,12,0,10.929529,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1
20215254,1067,0,5.445312,12,0,11.741661,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1
20215255,1068,0,4.050781,12,0,11.634754,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1


In [70]:
full_train_df.tail()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,sea_level_pressure,wind_direction,wind_speed,offset,hour,day,weekday,month,year,is_holiday
20215251,1064,0,5.042969,12,0,11.784570,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1
20215252,1065,0,1.827148,12,6,9.405249,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1
20215253,1066,0,3.291016,12,0,10.929529,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1
20215254,1067,0,5.445312,12,0,11.741661,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1
20215255,1068,0,4.050781,12,0,11.634754,1968.0,4.0,6.5,8.0,...,1019.5,340.0,6.0,0.0,23,31,5,12,116,1


In [71]:
## Reducing memory
full_train_df = reduce_mem_usage(full_train_df)
full_test_df = reduce_mem_usage(full_test_df)
gc.collect()

Mem. usage decreased to 810.58 Mb (41.1% reduction)
Mem. usage decreased to 1789.47 Mb (40.0% reduction)


0

In [72]:
beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

for item in beaufort:
    full_train_df.loc[(full_train_df['wind_speed']>=item[1]) & (full_train_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

In [73]:
full_train_df['group'] = full_train_df['month']
full_train_df['group'].replace((1,2,3,4,5,6), 1,inplace=True)
full_train_df['group'].replace((7,8,9,10,11,12), 2, inplace=True)

In [74]:
full_train_df = encode_cyclic_feature(full_train_df, 'weekday', 7)
full_train_df = encode_cyclic_feature(full_train_df, 'hour', 24)
full_train_df = encode_cyclic_feature(full_train_df, 'day', 31)
full_train_df = encode_cyclic_feature(full_train_df, 'month', 12)

In [75]:
full_train_df = reduce_mem_usage(full_train_df)

Mem. usage decreased to 942.54 Mb (37.5% reduction)


In [76]:
categoricals = ['site_id', 'building_id', 'primary_use', 'meter',  'wind_direction', 'is_holiday']
drop_cols = ['sea_level_pressure', 'wind_speed']
numericals = ['square_feet', 'year_built', 'air_temperature', 'cloud_coverage',
              'dew_temperature', 'precip_depth_1_hr', 'floor_count', 'beaufort_scale', 'weekday_sin', 'day_sin', 'hour_sin', 'month_sin']

feat_cols = categoricals + numericals

In [77]:
full_train_df.tail()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,wind_speed,offset,year,is_holiday,beaufort_scale,group,weekday_sin,hour_sin,day_sin,month_sin
20215251,1064,0,5.042969,12,0,11.781250,1968.0,4.0,6.5,8.0,...,6.0,0.0,116,1,4.0,2,-0.975098,-0.258789,-0.0,-0.0
20215252,1065,0,1.827148,12,6,9.406250,1968.0,4.0,6.5,8.0,...,6.0,0.0,116,1,4.0,2,-0.975098,-0.258789,-0.0,-0.0
20215253,1066,0,3.291016,12,0,10.929688,1968.0,4.0,6.5,8.0,...,6.0,0.0,116,1,4.0,2,-0.975098,-0.258789,-0.0,-0.0
20215254,1067,0,5.445312,12,0,11.742188,1968.0,4.0,6.5,8.0,...,6.0,0.0,116,1,4.0,2,-0.975098,-0.258789,-0.0,-0.0
20215255,1068,0,4.050781,12,0,11.632812,1968.0,4.0,6.5,8.0,...,6.0,0.0,116,1,4.0,2,-0.975098,-0.258789,-0.0,-0.0


In [78]:
full_train_df = reduce_mem_usage(full_train_df)
gc.collect()

Mem. usage decreased to 942.54 Mb (0.0% reduction)


0

In [79]:
full_test_df.head()

,row_id,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,sea_level_pressure,wind_direction,wind_speed,offset,hour,day,weekday,month,year,is_holiday
0,0,0,0,0,0,8.914062,2008.0,4.0,15.601562,2.0,...,1022.0,130.0,2.099609,5.0,0,1,6,1,117,0
1,1,1,0,0,0,7.910156,2004.0,4.0,15.601562,2.0,...,1022.0,130.0,2.099609,5.0,0,1,6,1,117,0
2,2,2,0,0,0,8.585938,1991.0,4.0,15.601562,2.0,...,1022.0,130.0,2.099609,5.0,0,1,6,1,117,0
3,3,3,0,0,0,10.070312,2002.0,4.0,15.601562,2.0,...,1022.0,130.0,2.099609,5.0,0,1,6,1,117,0
4,4,4,0,0,0,11.664062,1975.0,4.0,15.601562,2.0,...,1022.0,130.0,2.099609,5.0,0,1,6,1,117,0


In [80]:
target = full_train_df["meter_reading"]
del full_train_df["meter_reading"]

In [81]:
full_train_df.drop(drop_cols, axis=1)
gc.collect()

0

In [82]:
# Save the testing dataset to freeup the RAM. We'll read after training
# full_test_df.to_pickle('full_test_df.pkl')
# del full_test_df
# gc.collect()

In [83]:
full_train_df.head()

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,...,wind_speed,offset,year,is_holiday,beaufort_scale,group,weekday_sin,hour_sin,day_sin,month_sin
103,105,0,1,0,10.835938,1968.0,5.0,3.800781,0.0,2.400391,...,3.099609,0.0,116,1,2.0,1,-0.433838,0.0,0.201294,0.5
104,106,0,1,0,8.585938,1968.0,4.0,3.800781,0.0,2.400391,...,3.099609,0.0,116,1,2.0,1,-0.433838,0.0,0.201294,0.5
105,106,3,1,0,8.585938,1968.0,4.0,3.800781,0.0,2.400391,...,3.099609,0.0,116,1,2.0,1,-0.433838,0.0,0.201294,0.5
106,107,0,1,0,11.484375,2005.0,10.0,3.800781,0.0,2.400391,...,3.099609,0.0,116,1,2.0,1,-0.433838,0.0,0.201294,0.5
107,108,0,1,0,11.312500,1913.0,5.0,3.800781,0.0,2.400391,...,3.099609,0.0,116,1,2.0,1,-0.433838,0.0,0.201294,0.5


In [84]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.4,
            'subsample_freq': 1,
            'learning_rate': 0.25,
            'num_leaves': 31,
            'feature_fraction': 0.8,
            'lambda_l1': 1,
            'lambda_l2': 1
            }

folds = 2
seed = 666


kf = GroupKFold(n_splits=folds)

models = []
for train_index, val_index in kf.split(full_train_df, full_train_df['building_id'], groups=full_train_df['group']):

    train_X = full_train_df[feat_cols].iloc[train_index]
    val_X = full_train_df[feat_cols].iloc[val_index]
    train_y = target.iloc[train_index]
    val_y = target.iloc[val_index]
    lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
    lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)
    models.append(gbm)

/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.281496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3036
[LightGBM] [Info] Number of data points in the train set: 9498765, number of used features: 18


/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 4.184564
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.856447	valid_1's rmse: 1.18687
[200]	training's rmse: 0.799235	valid_1's rmse: 1.1795
[300]	training's rmse: 0.767202	valid_1's rmse: 1.17776
Early stopping, best iteration is:
[254]	training's rmse: 0.781538	valid_1's rmse: 1.17685


/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.318905 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3077
[LightGBM] [Info] Number of data points in the train set: 10267670, number of used features: 18


/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/diegooliveirafaria/.pyenv/versions/3.8.12/envs/sep/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 4.247750
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.882636	valid_1's rmse: 1.16912
[200]	training's rmse: 0.817825	valid_1's rmse: 1.16785
Early stopping, best iteration is:
[132]	training's rmse: 0.857951	valid_1's rmse: 1.16659


In [85]:
del full_train_df, train_X, val_X, lgb_train, lgb_eval, train_y, val_y, target
gc.collect()

727

In [86]:
# full_test_df = pd.read_pickle('full_test_df.pkl')

In [87]:
# full_test_df.loc[(full_test_df['primary_use'] == le.transform(['Education'])[0]) & (full_test_df['month'] >= 6) & (full_test_df['month'] <= 8), 'is_vacation_month'] = np.int8(1)
# full_test_df.loc[full_test_df['is_vacation_month']!=1, 'is_vacation_month'] = np.int8(0)

In [88]:
for item in beaufort:
    full_test_df.loc[(full_test_df['wind_speed']>=item[1]) & (full_test_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

In [89]:
full_test_df = encode_cyclic_feature(full_test_df, 'weekday', 7)
full_test_df = encode_cyclic_feature(full_test_df, 'hour', 24)
full_test_df = encode_cyclic_feature(full_test_df, 'day', 31)
full_test_df = encode_cyclic_feature(full_test_df, 'month', 12)

In [90]:
full_test_df = full_test_df[feat_cols]

In [91]:
full_test_df.head()

,site_id,building_id,primary_use,meter,wind_direction,is_holiday,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,floor_count,beaufort_scale,weekday_sin,day_sin,hour_sin,month_sin
0,0,0,0,0,130.0,0,8.914062,2008.0,15.601562,2.0,12.796875,0.0,4.0,2.0,-0.781831,0.201299,0.0,0.5
1,0,1,0,0,130.0,0,7.910156,2004.0,15.601562,2.0,12.796875,0.0,4.0,2.0,-0.781831,0.201299,0.0,0.5
2,0,2,0,0,130.0,0,8.585938,1991.0,15.601562,2.0,12.796875,0.0,4.0,2.0,-0.781831,0.201299,0.0,0.5
3,0,3,0,0,130.0,0,10.070312,2002.0,15.601562,2.0,12.796875,0.0,4.0,2.0,-0.781831,0.201299,0.0,0.5
4,0,4,0,0,130.0,0,11.664062,1975.0,15.601562,2.0,12.796875,0.0,4.0,2.0,-0.781831,0.201299,0.0,0.5


In [92]:
full_test_df = reduce_mem_usage(full_test_df)

Mem. usage decreased to 1590.64 Mb (42.9% reduction)


In [93]:
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(full_test_df.shape[0]/50000)))):
    res.append(np.expm1(sum([model.predict(full_test_df.iloc[i:i+step_size]) for model in models])/folds))
    i+=step_size

100%|█████████████████████████████████████████| 834/834 [02:10<00:00,  6.40it/s]


In [94]:
res

[array([123.92944772,  60.65646731,  11.97019202, ..., 143.32754208,
        213.98119333, 137.7267691 ]),
 array([ 99.85294047, 113.44267178, 129.42126899, ...,  82.78233851,
        952.7620451 , 121.51363749]),
 array([367.71765239,  46.69979567,  20.05469427, ..., 151.30565133,
          6.76357805,  26.14989936]),
 array([126.02893795,  56.80732351,  59.21631527, ..., 342.74338263,
        320.04498596, 232.08323602]),
 array([248.78483328, 738.11930365, 133.76393893, ..., 115.95790709,
        136.21775986, 100.95115729]),
 array([354.12023025, 375.89683482, 218.56605497, ..., 178.63623643,
        256.25321332, 163.25203921]),
 array([ 131.75931752,  142.5956411 ,  149.27214183, ..., 1078.31773765,
         588.40925636,   82.45777353]),
 array([1127.19379361,  132.58769256,  435.15173109, ...,   16.2467012 ,
          53.88445834,  125.13095464]),
 array([  5.41982344,  21.54371341, 124.44902053, ..., 153.8316946 ,
        248.59361724, 243.65110043]),
 array([182.36972645, 195

In [95]:
res = np.concatenate(res)

In [85]:
submission = pd.read_csv('../raw_data/sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_fe_lgbm.csv', index=False)
submission

,row_id,meter_reading
0,0,123.929448
1,1,60.656467
2,2,11.970192
3,3,198.892692
4,4,747.524203
...,...,...
41697595,41697595,4.665242
41697596,41697596,3.754255
41697597,41697597,1.327924
41697598,41697598,153.366725


## Save and Load the Model

### Save

In [88]:
models[0].save_model('model1.txt')
models[1].save_model('model2.txt')

### Load

In [177]:
model1,model2

NameError: name 'model1' is not defined

In [180]:
model2

NameError: name 'model2' is not defined

In [181]:
model11 = lgb.Booster(model_file='model1.txt')
model12 = lgb.Booster(model_file='model2.txt')

## Preprocess

In [205]:
X = pd.DataFrame(full_test_df.iloc[2:3,:])
X

,site_id,building_id,primary_use,meter,wind_direction,is_holiday,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,floor_count,beaufort_scale,weekday_sin,day_sin,hour_sin,month_sin
2,0,2,0,0,130.0,0,8.585938,1991.0,15.601562,2.0,12.796875,0.0,4.0,2.0,-0.781738,0.201294,0.0,0.5


In [210]:
def preprocess(X):
    model1 = lgb.Booster(model_file='model1.txt')
    model2 = lgb.Booster(model_file='model2.txt')
    y_pred = (model1.predict(X)+model2.predict(X))/2
    
    to_return = {'meter_reading':float(y_pred)}

    return to_return

In [211]:
print(preprocess(X))

{'meter_reading': 2.5626538030527133}


In [151]:
weather_preproc = pd.read_csv("../raw_data/weather_preproc.csv",parse_dates=['timestamp'])

In [145]:
building_metadata =  pd.read_csv("../raw_data/building_metadata.csv")

In [144]:
X

,row_id,building_id,meter,timestamp
2,2,2,0,2017-01-01


In [147]:
X = X.merge(building_metadata,on=['building_id'],how='left')

In [ ]:
(1,'Electricity','2020---f')

In [149]:
X.dtypes

row_id                  int64
building_id             int64
meter                   int64
timestamp      datetime64[ns]
site_id                 int64
primary_use            object
square_feet             int64
year_built            float64
floor_count           float64
dtype: object

In [152]:
X.merge(weather_preproc,on=['site_id','timestamp'],how='left')

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,Unnamed: 0,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset
0,2,2,0,2017-01-01,0,Education,5376,1991.0,NaN,5,15.6,2.0,12.8,0.0,1022.0,130.0,2.1,5


In [153]:
full_test_df

,site_id,building_id,primary_use,meter,wind_direction,is_holiday,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,floor_count,beaufort_scale,weekday_sin,day_sin,hour_sin,month_sin
0,0,0,0,0,130.0,0,8.914062,2008.0,15.601562,2.0,12.796875,0.000000,4.0,2.0,-0.781738,0.201294,0.00000,0.5
1,0,1,0,0,130.0,0,7.910156,2004.0,15.601562,2.0,12.796875,0.000000,4.0,2.0,-0.781738,0.201294,0.00000,0.5
2,0,2,0,0,130.0,0,8.585938,1991.0,15.601562,2.0,12.796875,0.000000,4.0,2.0,-0.781738,0.201294,0.00000,0.5
3,0,3,0,0,130.0,0,10.070312,2002.0,15.601562,2.0,12.796875,0.000000,4.0,2.0,-0.781738,0.201294,0.00000,0.5
4,0,4,0,0,130.0,0,11.664062,1975.0,15.601562,2.0,12.796875,0.000000,4.0,2.0,-0.781738,0.201294,0.00000,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41697595,15,1444,1,0,180.0,0,9.882812,1914.0,18.906250,0.0,10.601562,1.255859,4.0,2.0,0.975098,0.968262,0.96582,0.5
41697596,15,1445,0,0,180.0,0,8.367188,1968.0,18.906250,0.0,10.601562,1.255859,4.0,2.0,0.975098,0.968262,0.96582,0.5
41697597,15,1446,1,0,180.0,0,9.328125,1997.0,18.906250,0.0,10.601562,1.255859,4.0,2.0,0.975098,0.968262,0.96582,0.5
41697598,15,1447,4,0,180.0,0,10.304688,2001.0,18.906250,0.0,10.601562,1.255859,4.0,2.0,0.975098,0.968262,0.96582,0.5


In [163]:
pd.DataFrame(missing_features)

,0
year_built,1968.344315
floor_count,4.175326
cloud_coverage,2.750034
precip_depth_1_hr,1.255560
sea_level_pressure,1016.059575


In [168]:
missing_features.keys()

Index(['year_built', 'floor_count', 'cloud_coverage', 'precip_depth_1_hr',
       'sea_level_pressure'],
      dtype='object')

In [197]:
test_df

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00
...,...,...,...,...
41697595,41697595,1444,0,2018-05-09 07:00:00
41697596,41697596,1445,0,2018-05-09 07:00:00
41697597,41697597,1446,0,2018-05-09 07:00:00
41697598,41697598,1447,0,2018-05-09 07:00:00


In [215]:
full_test_df.columns

Index(['site_id', 'building_id', 'primary_use', 'meter', 'wind_direction',
       'is_holiday', 'square_feet', 'year_built', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'floor_count',
       'beaufort_scale', 'weekday_sin', 'day_sin', 'hour_sin', 'month_sin'],
      dtype='object')

In [216]:
weather_preproc.isnull().sum()

Unnamed: 0                 0
site_id                    0
timestamp                  0
air_temperature          104
cloud_coverage        140448
dew_temperature          327
precip_depth_1_hr      95588
sea_level_pressure     21265
wind_direction         12370
wind_speed               460
offset                     0
dtype: int64